In [1]:
import os
import torch
import GPUtil
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [ ]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/sigmoid_nonlinear/sigmoid_nonlinear.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'transfer_learning/results/sigmoid_nonlinear')
MODEL_RESULTS_AE_PATH = os.path.join(ROOT_PATH, r'transfer_learning/results/sigmoid_nonlinear/model_autoencoder_AE')
MODEL_RESULTS_PGNNIV_PATH = os.path.join(ROOT_PATH, r'transfer_learning/results/sigmoid_nonlinear/model_autoencoder_NN')


# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_AE_PATH)
create_folder(MODEL_RESULTS_PGNNIV_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/transfer_learning/results/non_linear
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/transfer_learning/results/sigmoid_nonlinear/model_autoencoder_AE
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/transfer_learning/results/sigmoid_nonlinear/model_autoencoder_NN


In [3]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/sigmoid_nonlinear/sigmoid_nonlinear.pkl


In [4]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [5]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


### División de los datos

In [6]:
X_train = torch.Tensor(dataset['X_train']).unsqueeze(1)
y_train = torch.Tensor(dataset['y_train']).unsqueeze(1)
K_train = torch.tensor(dataset['k_train']).unsqueeze(1)
f_train = torch.tensor(dataset['f_train']).unsqueeze(1).to(torch.float32)

X_val = torch.Tensor(dataset['X_val']).unsqueeze(1)
y_val = TensOps(torch.Tensor(dataset['y_val']).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val']).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val']).to(torch.float32).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

print("Train dataset length:", len(X_train))
print("Validation dataset length:", len(X_val))

Train dataset length: 80
Validation dataset length: 20


In [7]:
N_data_AE = len(X_train)//2
N_data_NN = len(X_train) - len(X_train)//2
prop_data_NN = 1 - N_data_AE/(N_data_NN + N_data_AE)

print("Dataset length for the autoencoder:", N_data_AE)
print("Dataset length for the PGNNIV:", N_data_NN)

X_AE, X_NN, y_AE, y_NN, K_AE, K_NN, f_AE, f_NN = train_test_split(X_train, y_train, K_train, f_train, test_size=prop_data_NN, random_state=42)

Dataset length for the autoencoder: 40
Dataset length for the PGNNIV: 40


#### Datos para el autoencoder

In [8]:
y_train_AE, y_test_AE = train_test_split(y_AE, test_size=0.2, random_state=42)

y_train_AE = TensOps(y_train_AE.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
y_test_AE = TensOps(y_test_AE.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

#### Datos para la PGNNIV

In [9]:
X_train_NN, X_test_NN, y_train_NN, y_test_NN, K_train_NN, K_test_NN, f_train_NN, f_test_NN = train_test_split(X_NN, y_NN, K_NN, f_NN, test_size=0.2, random_state=42)

X_train_NN = X_train_NN.to(DEVICE)
X_test_NN = X_test_NN.to(DEVICE)

y_train_NN = TensOps(y_train_NN.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
y_test_NN = TensOps(y_test_NN.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

K_train_NN = TensOps(K_train_NN.to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
K_test_NN = TensOps(K_test_NN.to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

f_train_NN = TensOps(f_train_NN.to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
f_test_NN = TensOps(f_test_NN.to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

## Autoencoder

In [10]:
from model import Autoencoder
from trainers.train import train_autoencoder_loop

In [11]:
n_modes = 10

autoencoder_input_shape = y_train_AE.values[0].shape
latent_space_dim = [20, 10, n_modes, 10, 20]
autoencoder_output_shape = y_train_AE.values[0].shape

In [12]:
X_train = y_train_AE.values
y_train = y_train_AE

X_test = y_test_AE.values
y_test = y_test_AE

In [13]:
autoencoder = Autoencoder(autoencoder_input_shape, latent_space_dim, autoencoder_output_shape).to(DEVICE)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)

start_epoch = 0
n_epochs = 10000
batch_size = 64
n_checkpoint = 3
new_lr = None

train_autoencoder_loop(autoencoder, optimizer, X_train, y_train, X_test, y_test,  
                       n_checkpoint, start_epoch, n_epochs, batch_size, MODEL_RESULTS_AE_PATH, DEVICE, new_lr)

Starting training from scratch.
Epoch 0, Train loss: 6.113e+01, Test loss: 1.006e+02
Epoch 100, Train loss: 8.462e+00, Test loss: 1.261e+01
Epoch 200, Train loss: 6.199e+00, Test loss: 1.087e+01
Epoch 300, Train loss: 5.323e+00, Test loss: 1.106e+01
Epoch 400, Train loss: 3.396e+00, Test loss: 1.083e+01
Epoch 500, Train loss: 2.946e+00, Test loss: 1.000e+01
Epoch 600, Train loss: 2.425e+00, Test loss: 7.302e+00
Epoch 700, Train loss: 1.892e+00, Test loss: 4.614e+00
Epoch 800, Train loss: 1.421e+00, Test loss: 3.764e+00
Epoch 900, Train loss: 1.298e+00, Test loss: 3.564e+00
Epoch 1000, Train loss: 1.205e+00, Test loss: 3.353e+00
Epoch 1100, Train loss: 1.129e+00, Test loss: 3.139e+00
Epoch 1200, Train loss: 1.058e+00, Test loss: 3.005e+00
Epoch 1300, Train loss: 1.002e+00, Test loss: 2.885e+00
Epoch 1400, Train loss: 9.556e-01, Test loss: 2.767e+00
Epoch 1500, Train loss: 9.161e-01, Test loss: 2.651e+00
Epoch 1600, Train loss: 8.852e-01, Test loss: 2.536e+00
Epoch 1700, Train loss: 8.53

In [14]:
start_epoch = 9999
n_epochs = 20000
batch_size = 64
n_checkpoint = 3
new_lr = 1e-4

train_autoencoder_loop(autoencoder, optimizer, X_train, y_train, X_test, y_test,  
                       n_checkpoint, start_epoch, n_epochs, batch_size, MODEL_RESULTS_AE_PATH, DEVICE, new_lr)

Starting training from a checkpoint. Epoch 9999.
Epoch 10000, Train loss: 2.289e-01, Test loss: 8.856e-01
Epoch 10100, Train loss: 2.288e-01, Test loss: 8.854e-01
Epoch 10200, Train loss: 2.288e-01, Test loss: 8.851e-01
Epoch 10300, Train loss: 2.287e-01, Test loss: 8.849e-01
Epoch 10400, Train loss: 2.287e-01, Test loss: 8.846e-01
Epoch 10500, Train loss: 2.286e-01, Test loss: 8.844e-01
Epoch 10600, Train loss: 2.286e-01, Test loss: 8.841e-01
Epoch 10700, Train loss: 2.285e-01, Test loss: 8.838e-01
Epoch 10800, Train loss: 2.285e-01, Test loss: 8.836e-01
Epoch 10900, Train loss: 2.284e-01, Test loss: 8.833e-01
Epoch 11000, Train loss: 2.283e-01, Test loss: 8.829e-01
Epoch 11100, Train loss: 2.283e-01, Test loss: 8.826e-01
Epoch 11200, Train loss: 2.282e-01, Test loss: 8.823e-01
Epoch 11300, Train loss: 2.281e-01, Test loss: 8.820e-01
Epoch 11400, Train loss: 2.280e-01, Test loss: 8.816e-01
Epoch 11500, Train loss: 2.279e-01, Test loss: 8.812e-01
Epoch 11600, Train loss: 2.278e-01, Tes

## PGNNIV

In [15]:
from vecopsciml.operators.zero_order import Mx, My
from model.ae_nonlinear_model import AutoencoderNonlinearModel

In [16]:
# Predictive network architecture
input_shape = X_train_NN[0].shape
predictive_layers = [20, 10, n_modes]
predictive_output = y_train_NN.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train_NN)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train_NN)).values[0].shape

# Other parameters
n_filters_explanatory = 5

In [17]:
pretrained_decoder = autoencoder.decoder

for param in pretrained_decoder.parameters():
    param.requires_grad = False

for name, param in pretrained_decoder.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

hidden1_layer.weight: requires_grad=False
hidden1_layer.bias: requires_grad=False
hidden2_layer.weight: requires_grad=False
hidden2_layer.bias: requires_grad=False
output_layer.weight: requires_grad=False
output_layer.bias: requires_grad=False


In [18]:
model = AutoencoderNonlinearModel(input_shape, predictive_layers, pretrained_decoder, predictive_output, explanatory_input,
                                   explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 20000

batch_size = 64
n_checkpoints = 10

train_loop(model, optimizer, X_train_NN, y_train_NN, f_train_NN, X_test_NN, y_test_NN, f_test_NN,
           D, n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PGNNIV_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 3.140e+11, Test loss: 4.405e+11, MSE(e): 3.138e+01, MSE(pi1): 7.508e+00, MSE(pi2): 1.347e+01, MSE(pi3): 1.068e+00
Epoch 100, Train loss: 6.414e+10, Test loss: 5.539e+10, MSE(e): 6.400e+00, MSE(pi1): 7.944e+00, MSE(pi2): 2.822e+00, MSE(pi3): 6.284e-01
Epoch 200, Train loss: 5.739e+10, Test loss: 5.002e+10, MSE(e): 5.726e+00, MSE(pi1): 7.661e+00, MSE(pi2): 2.534e+00, MSE(pi3): 5.946e-01
Epoch 300, Train loss: 4.742e+10, Test loss: 4.276e+10, MSE(e): 4.730e+00, MSE(pi1): 6.797e+00, MSE(pi2): 2.113e+00, MSE(pi3): 5.170e-01
Epoch 400, Train loss: 3.147e+10, Test loss: 3.079e+10, MSE(e): 3.139e+00, MSE(pi1): 4.246e+00, MSE(pi2): 1.471e+00, MSE(pi3): 3.270e-01
Epoch 500, Train loss: 1.907e+10, Test loss: 2.341e+10, MSE(e): 1.903e+00, MSE(pi1): 2.588e+00, MSE(pi2): 9.877e-01, MSE(pi3): 2.091e-01
Epoch 600, Train loss: 1.329e+10, Test loss: 1.930e+10, MSE(e): 1.325e+00, MSE(pi1): 2.397e+00, MSE(pi2): 7.247e-01, MSE(pi3): 1.707e-01
Epoch 700, 

In [20]:
# Parametros de entrenamiento
start_epoch = 18000
n_epochs = 100000

batch_size = 64 
n_checkpoints = 100

second_lr = 1e-4

train_loop(model, optimizer, X_train_NN, y_train_NN, f_train_NN, X_test_NN, y_test_NN, f_test_NN,
           D, n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PGNNIV_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 18000.
Epoch 18000, Train loss: 2.253e+06, Test loss: 2.289e+08, MSE(e): 9.265e-05, MSE(pi1): 9.732e-02, MSE(pi2): 4.641e-05, MSE(pi3): 3.535e-03
Epoch 18100, Train loss: 2.252e+06, Test loss: 2.289e+08, MSE(e): 9.259e-05, MSE(pi1): 9.731e-02, MSE(pi2): 4.638e-05, MSE(pi3): 3.534e-03
Epoch 18200, Train loss: 2.252e+06, Test loss: 2.288e+08, MSE(e): 9.253e-05, MSE(pi1): 9.729e-02, MSE(pi2): 4.634e-05, MSE(pi3): 3.534e-03
Epoch 18300, Train loss: 2.251e+06, Test loss: 2.288e+08, MSE(e): 9.248e-05, MSE(pi1): 9.728e-02, MSE(pi2): 4.630e-05, MSE(pi3): 3.534e-03
Epoch 18400, Train loss: 2.250e+06, Test loss: 2.287e+08, MSE(e): 9.241e-05, MSE(pi1): 9.726e-02, MSE(pi2): 4.626e-05, MSE(pi3): 3.534e-03
Epoch 18500, Train loss: 2.249e+06, Test loss: 2.286e+08, MSE(e): 9.234e-05, MSE(pi1): 9.724e-02, MSE(pi2): 4.622e-05, MSE(pi3): 3.534e-03
Epoch 18600, Train loss: 2.248e+06, Test loss: 2.285e+08, MSE(e): 9.227e-05, MSE(pi1): 9.722e-02, MSE(pi2): 4.618e-0